# Imports

In [1]:
%run PII_Util.py

D:\Anaconda3\envs\nlp_torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Data

In [2]:
path_train = './in/train.json'
path_test = './in/test.json'

In [3]:
train_json = json.load(open(path_train))
df_train = pd.json_normalize(train_json)

test_json = json.load(open(path_test))
df_test = pd.json_normalize(test_json)

In [4]:
classes

['O',
 'B-EMAIL',
 'B-ID_NUM',
 'B-NAME_STUDENT',
 'B-PHONE_NUM',
 'B-STREET_ADDRESS',
 'B-URL_PERSONAL',
 'B-USERNAME',
 'I-ID_NUM',
 'I-NAME_STUDENT',
 'I-PHONE_NUM',
 'I-STREET_ADDRESS',
 'I-URL_PERSONAL']

In [5]:
classes2id

{'O': 0,
 'B-EMAIL': 1,
 'B-ID_NUM': 2,
 'B-NAME_STUDENT': 3,
 'B-PHONE_NUM': 4,
 'B-STREET_ADDRESS': 5,
 'B-URL_PERSONAL': 6,
 'B-USERNAME': 7,
 'I-ID_NUM': 8,
 'I-NAME_STUDENT': 9,
 'I-PHONE_NUM': 10,
 'I-STREET_ADDRESS': 11,
 'I-URL_PERSONAL': 12}

In [6]:
entities = ['O', 'EMAIL', 'ID_NUM', 'NAME_STUDENT', 'PHONE_NUM', 'STREET_ADDRESS', 'URL_PERSONAL', 'USERNAME']
entity2id = {entity:i for i, entity in enumerate(entities)}

entity2class = {entity:[f'B-{entity}',f'I-{entity}'] for entity in entities}
entity2class['O'] = ['O','O']
entity2class['EMAIL'][1] = entity2class['EMAIL'][0]
entity2class['ID_NUM'][1] = entity2class['ID_NUM'][0]
entity2class['USERNAME'][1] = entity2class['USERNAME'][0]


entity_id2class_id = {entity2id[entity]:[classes2id[label] for label in classes] for entity,classes in entity2class.items()}



np_entity_id2class_id = [classes for classes in entity_id2class_id.values()]

In [7]:
np_entity_id2class_id

[[0, 0], [1, 1], [2, 2], [3, 9], [4, 10], [5, 11], [6, 12], [7, 7]]

In [8]:
entity_id2class_id

{0: [0, 0],
 1: [1, 1],
 2: [2, 2],
 3: [3, 9],
 4: [4, 10],
 5: [5, 11],
 6: [6, 12],
 7: [7, 7]}

In [9]:
entity2class

{'O': ['O', 'O'],
 'EMAIL': ['B-EMAIL', 'B-EMAIL'],
 'ID_NUM': ['B-ID_NUM', 'B-ID_NUM'],
 'NAME_STUDENT': ['B-NAME_STUDENT', 'I-NAME_STUDENT'],
 'PHONE_NUM': ['B-PHONE_NUM', 'I-PHONE_NUM'],
 'STREET_ADDRESS': ['B-STREET_ADDRESS', 'I-STREET_ADDRESS'],
 'URL_PERSONAL': ['B-URL_PERSONAL', 'I-URL_PERSONAL'],
 'USERNAME': ['B-USERNAME', 'B-USERNAME']}

In [10]:
train_json[0].keys()

dict_keys(['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels'])

In [11]:
df_train.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [12]:
text_test = """Meet Jane Doe, a brilliant student at XYZ University. She can be reached at jane.doe@email.com or through her phone number +1234567890. Jane resides at 123 Main Street, Cityville. Her student ID is 987654 and her personal website is www.janedoe.com. Connect with her on social media using the username @janedoe.

Meanwhile, John Smith, another outstanding student, can be contacted at john.smith@email.com or at +9876543210. John lives at 456 Oak Avenue, Townsville. His student ID is 123456, and you can visit his personal blog at www.johnsmithblog.com. Follow him on Twitter with the handle @johnsmith123.

For any inquiries about the university's programs, you can contact the administration office at admin@xyzuniversity.edu or call +5551234567. The office is located at 789 University Boulevard.

Visit our official website at www.xyzuniversity.edu for more information on courses and admission procedures.

"""


In [13]:
text_test

"Meet Jane Doe, a brilliant student at XYZ University. She can be reached at jane.doe@email.com or through her phone number +1234567890. Jane resides at 123 Main Street, Cityville. Her student ID is 987654 and her personal website is www.janedoe.com. Connect with her on social media using the username @janedoe.\n\nMeanwhile, John Smith, another outstanding student, can be contacted at john.smith@email.com or at +9876543210. John lives at 456 Oak Avenue, Townsville. His student ID is 123456, and you can visit his personal blog at www.johnsmithblog.com. Follow him on Twitter with the handle @johnsmith123.\n\nFor any inquiries about the university's programs, you can contact the administration office at admin@xyzuniversity.edu or call +5551234567. The office is located at 789 University Boulevard.\n\nVisit our official website at www.xyzuniversity.edu for more information on courses and admission procedures.\n\n"

# Trainer API

## Instantiate

In [10]:
%%time
model_adapter = Yanis_Adapter(threshold = 0.1)
# text = text_test
# text = df_train.loc[0].full_text
# labels_true = df_train.loc[0].labels
# threshold = 0.1

CPU times: total: 5.61 s
Wall time: 6.71 s


In [11]:
model = model_adapter.model

## Dataset

In [16]:
import pandas as pd
from datasets import Dataset

# Convert the Pandas DataFrame to a Hugging Face dataset
# hf_dataset = Dataset.from_pandas(df_train.iloc[:100])
hf_dataset = Dataset.from_pandas(df_train)


In [17]:
hf_dataset

Dataset({
    features: ['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels'],
    num_rows: 6807
})

## Preprocessing

In [18]:
# def convert_to_features(sample):
#     return {
#         "input_ids": tokenizer(sample["text"], return_tensors="pt")["input_ids"].squeeze(),
#         "attention_mask": tokenizer(sample["text"], return_tensors="pt")["attention_mask"].squeeze(),
#         "labels": torch.tensor(sample["labels"])
#     }

# # Convert dataset to features
# features = [convert_to_features(sample) for sample in hf_dataset]

### Tokenize

In [19]:
%%time
def tokenize_function(example, model_adapter):
    global labels_aligned
    
    tokenizer = model_adapter.tokenizer
    cur_label2model_id = model_adapter.cur_label2model_id
    
    inputs = tokenizer(example["tokens"], return_tensors='pt', return_offsets_mapping=False, is_split_into_words=True)
    tokens = inputs.tokens()
    word_ids = inputs.word_ids()
    
    
    
#     return {
#         "input_ids": inputs["input_ids"].squeeze(),
#         "attention_mask": inputs["attention_mask"].squeeze(),
#         "tokens": tokens,
#         "words": example["tokens"],
#         "word_labels": example["labels"],
#         "word_ids": inputs.word_ids()
#     }

    labels = align_words(tokens, example["tokens"], word_ids, example["labels"])
    label_ids = [cur_label2model_id[label] for label in labels]
    labels_aligned = labels
    
    return {
        "input_ids": inputs["input_ids"].squeeze(),
        "attention_mask": inputs["attention_mask"].squeeze(),
        "tokens": tokens,
        "words": example["tokens"],
        "word_labels": example["labels"],
        "word_ids": inputs.word_ids(),
        "labels": torch.tensor(label_ids),
    }

# tokenized_datasets = hf_dataset.map(tokenize_function, fn_kwargs={"model_adapter": model_adapter}, batched=False)


# def tokenize_function(examples):
#     return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)

# tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

CPU times: total: 0 ns
Wall time: 0 ns


### Split

In [20]:
train_size = int(0.8 * len(hf_dataset))
eval_size = len(hf_dataset) - train_size
# train_dataset, eval_dataset = hf_dataset.train_test_split(test_size=eval_size)
# list_remove = ['labels']
# dataset_filtered = tokenized_datasets.remove_columns(list_remove)
dataset_filtered = tokenized_datasets

split_datasets = dataset_filtered.train_test_split(test_size=eval_size, seed = 42)
train_dataset = split_datasets['train']
eval_dataset = split_datasets['test']

NameError: name 'tokenized_datasets' is not defined

In [ ]:
train_dataset

In [ ]:
eval_dataset

## Data Collator

In [ ]:
data_collator = DataCollatorForTokenClassification(model_adapter.tokenizer)

## Metric and Postprocessing

In [24]:
def preprocess_logits(logits, label):
    np_probs = torch.softmax(logits, axis=-1) 
    
    return np_probs

In [25]:
import functools
from datasets import load_metric

In [26]:
METRIC_F1 = load_metric("f1",  beta=5)
METRIC_PRECISION = load_metric("precision")
METRIC_RECALL = load_metric("recall")

C:\Users\cloud\AppData\Local\Temp\ipykernel_16788\3877367123.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  METRIC_F1 = load_metric("f1",  beta=5)
D:\Anaconda3\envs\nlp_torch\Lib\site-packages\datasets\load.py:753: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
D:\Anaconda3\envs\nlp_torch\Lib\site-packages\datasets\load.py:753: FutureWarning: The repository for precision contains custom code which must be executed to correctly load t

In [27]:
    ## (Vectorized) Get highest non-'O' labels for each token with thresholding
    # Get 'O' index of model
    # Get argsort
    # Get First max
    # Get == 'O' Mask
    # Get threshold mask
    # Get final indices

In [28]:
def compute_metrics_base(eval_preds, model_adapter, threshold = 0.1):
    
    global eval_preds_copy
    
    
    eval_preds_copy = eval_preds
    
    np_probs, true_labels_id, inputs = eval_preds
#     logits, true_labels_id = eval_preds
#     np_probs = np.argmax(logits, axis=-1)
#     tokens = inputs.tokens()


    label2id =  model_adapter.model.config.label2id
    o_index = label2id['O']

    np_sorted_indices = np.argsort(np_probs)
    np_max_indices = np_sorted_indices[:,:, -1]
    # np_max = np_sorted_indices[np.arange(np_max_indices.shape[0]), np_max_indices]
    np_max_prob = np.take_along_axis(np_probs, np_max_indices[:, :, np.newaxis], axis=2).squeeze()

    np_2nd_max_indices = np_sorted_indices[:, :, -2]
    # np_2nd_max = np_sorted_indices[np.arange(np_sorted_indices.shape[0]), np_max_indices]
    np_2nd_max_prob = np.take_along_axis(np_probs, np_2nd_max_indices[:, :, np.newaxis], axis=2).squeeze()

    np_O_mask = np_max_indices == o_index
    np_threshold_mask = np_2nd_max_prob > threshold

    np_replace_mask = np_threshold_mask & np_O_mask

    np_label_ids = np.where(np_replace_mask, np_2nd_max_indices, np_max_indices)

    flat_label_ids = np_label_ids.flatten()
    flat_true_labels_id = true_labels_id.flatten()

    # Postprocess labels, convert irrelevant labels to 'O'
    np_labels_irrelevant = np.array(model_adapter.labels_irrelevant)
    flat_label_ids_mask = np.isin(flat_label_ids, np_labels_irrelevant)
    flat_label_ids[flat_label_ids_mask] = o_index

    # Also remove paddings
    mask_padding_inv = flat_true_labels_id != -100
    flat_true_labels_id = flat_true_labels_id[mask_padding_inv]
    flat_label_ids = flat_label_ids[mask_padding_inv]

    dict_scores = {}

    f1_score = METRIC_F1.compute(predictions=flat_label_ids, references=flat_true_labels_id, labels=classes_pos_id, average ='micro')
    f_beta_score = fbeta_score(y_true = flat_true_labels_id, y_pred = flat_label_ids , labels = classes_pos_id,beta=5, average='micro')
    precision = METRIC_PRECISION.compute(predictions=flat_label_ids, references=flat_true_labels_id,labels=classes_pos_id, average ='micro')
    recall = METRIC_RECALL.compute(predictions=flat_label_ids, references=flat_true_labels_id,labels=classes_pos_id, average ='micro')

    for score in [f1_score, precision, recall]:
        dict_scores.update(score) 

    dict_scores['f_beta'] = f_beta_score

    return dict_scores



#     #Test above first
    
    
#     ##Compute competition metrics
    
#     # Step 4: Create a list of which tokens or subwords correspond to a word using the word_ids variable
#     word_subword_mapping = {}
#     for i, word_id in enumerate(word_ids):
#         if word_id is not None:
#             if word_id not in word_subword_mapping:
#                 word_subword_mapping[word_id] = []
#             word_subword_mapping[word_id].append(i)

#     # Step 5: Iterate through pairs of words and subwords to count the majority label
#     word_labels = []
#     for i, word in enumerate(words):
#         if i in word_subword_mapping:
#             subword_labels = pred_labels[word_subword_mapping[i]]
#             majority_label = get_majority(subword_labels)
#             word_labels.append((word, majority_label))
#         else:
#             word_labels.append((word,'O'))


#     token_v, pred_v = zip(*word_labels)

#     pred_conv = [model_id2cur_label[model.config.label2id[pred]] for pred in pred_v]
#     pred_bio = convert_to_bio(token_v, pred_conv)

#     f_beta = fbeta_score(labels, pred_bio, labels = classes_pos,beta=5, average='micro')
    
    
    
#     return metric.compute(predictions=predictions, references=labels)


    #Threshold + postprocessing
    #
    #Compute precision, recall, f1_beta from preds and labels
    
    #
    
    #Get 
    #Get

In [26]:
compute_metrics = functools.partial(compute_metrics_base, model_adapter=model_adapter, threshold=0.1)

## Trainer API

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch
from datasets import load_metric

In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    per_device_eval_batch_size=4,    # batch size per device during evaluation
    logging_dir='./logs',            # directory for storing logs
#     do_eval=True                     # Perform evaluation
    include_inputs_for_metrics = True
)

# Initialize Trainer with the evaluation only mode
trainer = Trainer(
    model=model_adapter.model,       # the instantiated 🤗 Transformers model to be trained
    args=training_args,              # training arguments, defined above
    eval_dataset=eval_dataset,       # evaluation dataset
    data_collator=data_collator,     # data collator for evaluation
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics = preprocess_logits
)

In [25]:
%%time
pred_results = trainer.predict(eval_dataset.select(range(4)))

D:\Anaconda3\envs\nlp_torch\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


CPU times: total: 4min 21s
Wall time: 45.9 s


# Results Analysis

In [2]:
%%time
def pad_lists(lst, pad_value, max_length = None):
    if max_length is None:
        max_length = max(len(sublist) for sublist in lst)
    padded_lst = np.full((len(lst), max_length), pad_value)
    for i, sublist in enumerate(lst):
        padded_lst[i, :len(sublist)] = sublist
    return padded_lst

# Example list of lists with varying lengths
# list_of_lists = [[1, 2, 3], [4, 5], [6], [7, 8, 9, 10]]

# # Padding value
# pad_value = -100

# # Convert to numpy array with padding
# padded_array = pad_lists(tokenized_datasets['labels'], pad_value)
# padded_array.shape

CPU times: total: 0 ns
Wall time: 0 ns


In [52]:
# tokenized_datasets.save_to_disk("./in/tokenized_datasets")

Saving the dataset (1/1 shards): 100%|███████████████████████████████████| 6807/6807 [00:00<00:00, 23086.59 examples/s]


In [3]:
from datasets import load_dataset, load_from_disk

In [4]:
tokenized_datasets = load_from_disk("./in/tokenized_datasets")

In [5]:
%%time
loaded_predictions = np.load("./in/preds01.npy")

CPU times: total: 844 ms
Wall time: 20.2 s


In [13]:
%%time
labels_padded = pad_lists(tokenized_datasets['labels'], -100, loaded_predictions.shape[1])

CPU times: total: 3.94 s
Wall time: 3.94 s


In [6]:
tokenized_datasets

Dataset({
    features: ['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels', 'input_ids', 'attention_mask', 'words', 'word_labels', 'word_ids'],
    num_rows: 6807
})

In [44]:
%%time
eval_pred = (loaded_predictions,  labels_padded, None)

metrics_01 = compute_metrics((eval_pred)) 
metrics_01

CPU times: total: 2min 14s
Wall time: 2min 25s


{'f1': 0.18650248072209935,
 'precision': 0.10458567980691874,
 'recall': 0.8604522890237176,
 'f_beta': 0.6732955960226424}

In [49]:
%%time 
compute_metrics = functools.partial(compute_metrics_base, model_adapter=model_adapter, threshold=0.50)
eval_pred = (loaded_predictions, labels_padded, None)

metrics_50 = compute_metrics((eval_pred)) 
metrics_50

CPU times: total: 2min 7s
Wall time: 2min 15s


{'f1': 0.30883490639022126,
 'precision': 0.18993846349045232,
 'recall': 0.8257032542746828,
 'f_beta': 0.7315271630345916}

In [50]:
%%time 
compute_metrics = functools.partial(compute_metrics_base, model_adapter=model_adapter, threshold=0.95)
eval_pred = (loaded_predictions, labels_padded, None)

metrics_95 = compute_metrics((eval_pred)) 
metrics_95

CPU times: total: 2min 12s
Wall time: 2min 24s


{'f1': 0.30883490639022126,
 'precision': 0.18993846349045232,
 'recall': 0.8257032542746828,
 'f_beta': 0.7315271630345916}

In [51]:
%%time 
compute_metrics = functools.partial(compute_metrics_base, model_adapter=model_adapter, threshold=0.3)
eval_pred = (loaded_predictions, labels_padded, None)

metrics_30 = compute_metrics((eval_pred)) 
metrics_30

CPU times: total: 2min 13s
Wall time: 2min 26s


{'f1': 0.30844354018311293,
 'precision': 0.18909816639640764,
 'recall': 0.8361831218974076,
 'f_beta': 0.7389299238873683}

In [52]:
%%time 
compute_metrics = functools.partial(compute_metrics_base, model_adapter=model_adapter, threshold=0.2)
eval_pred = (loaded_predictions, labels_padded, None)

metrics_20 = compute_metrics((eval_pred)) 
metrics_20

CPU times: total: 2min 7s
Wall time: 2min 20s


{'f1': 0.30343737653101543,
 'precision': 0.18481530501744675,
 'recall': 0.8472145615002757,
 'f_beta': 0.7445745394884034}

In [53]:
%%time 
compute_metrics = functools.partial(compute_metrics_base, model_adapter=model_adapter, threshold=0.15)
eval_pred = (loaded_predictions, labels_padded, None)

metrics_15 = compute_metrics((eval_pred)) 
metrics_15

CPU times: total: 2min 8s
Wall time: 2min 16s


{'f1': 0.2792207792207792,
 'precision': 0.16690026954177897,
 'recall': 0.8538334252619967,
 'f_beta': 0.7371428571428571}

In [54]:
%%time 
compute_metrics = functools.partial(compute_metrics_base, model_adapter=model_adapter, threshold=0.25)
eval_pred = (loaded_predictions, labels_padded, None)

metrics_25 = compute_metrics((eval_pred)) 
metrics_25

CPU times: total: 2min 7s
Wall time: 2min 13s


{'f1': 0.3078939409214639,
 'precision': 0.18837897853441896,
 'recall': 0.8422504136789851,
 'f_beta': 0.7430517864161255}

In [55]:
%%time 
compute_metrics = functools.partial(compute_metrics_base, model_adapter=model_adapter, threshold=0.18)
eval_pred = (loaded_predictions, labels_padded, None)

metrics_18 = compute_metrics((eval_pred)) 
metrics_18

CPU times: total: 2min 3s
Wall time: 2min 8s


{'f1': 0.30017044071098126,
 'precision': 0.18226979714944705,
 'recall': 0.8499724214009928,
 'f_beta': 0.7450050669864912}

## Retry

In [34]:
# Previous results are slightly incorrect because they use classes_pos_id while labels_padded is based on model labels
# Should use model_adapter.label_pos_ids (not previously implemented)

In [31]:
def compute_metrics_base(eval_preds, model_adapter, threshold = 0.1):
    
    global eval_preds_copy
    
    
    eval_preds_copy = eval_preds
    
    np_probs, true_labels_id, inputs = eval_preds
#     logits, true_labels_id = eval_preds
#     np_probs = np.argmax(logits, axis=-1)
#     tokens = inputs.tokens()


    label2id =  model_adapter.model.config.label2id
    o_index = label2id['O']

    np_sorted_indices = np.argsort(np_probs)
    np_max_indices = np_sorted_indices[:,:, -1]
    # np_max = np_sorted_indices[np.arange(np_max_indices.shape[0]), np_max_indices]
    np_max_prob = np.take_along_axis(np_probs, np_max_indices[:, :, np.newaxis], axis=2).squeeze()

    np_2nd_max_indices = np_sorted_indices[:, :, -2]
    # np_2nd_max = np_sorted_indices[np.arange(np_sorted_indices.shape[0]), np_max_indices]
    np_2nd_max_prob = np.take_along_axis(np_probs, np_2nd_max_indices[:, :, np.newaxis], axis=2).squeeze()

    np_O_mask = np_max_indices == o_index
    np_threshold_mask = np_2nd_max_prob > threshold

    np_replace_mask = np_threshold_mask & np_O_mask

    np_label_ids = np.where(np_replace_mask, np_2nd_max_indices, np_max_indices)

    flat_label_ids = np_label_ids.flatten()
    flat_true_labels_id = true_labels_id.flatten()

    # Postprocess labels, convert irrelevant labels to 'O'
    np_labels_irrelevant = np.array(model_adapter.labels_irrelevant)
    flat_label_ids_mask = np.isin(flat_label_ids, np_labels_irrelevant)
    flat_label_ids[flat_label_ids_mask] = o_index

    # Also remove paddings
    mask_padding_inv = flat_true_labels_id != -100
    flat_true_labels_id = flat_true_labels_id[mask_padding_inv]
    flat_label_ids = flat_label_ids[mask_padding_inv]

    dict_scores = {}

    f1_score = METRIC_F1.compute(predictions=flat_label_ids, references=flat_true_labels_id, labels=model_adapter.label_pos_ids, average ='micro')
    f_beta_score = fbeta_score(y_true = flat_true_labels_id, y_pred = flat_label_ids , labels = model_adapter.label_pos_ids,beta=5, average='micro')
    precision = METRIC_PRECISION.compute(predictions=flat_label_ids, references=flat_true_labels_id,labels=model_adapter.label_pos_ids, average ='micro')
    recall = METRIC_RECALL.compute(predictions=flat_label_ids, references=flat_true_labels_id,labels=model_adapter.label_pos_ids, average ='micro')

    for score in [f1_score, precision, recall]:
        dict_scores.update(score) 

    dict_scores['f_beta'] = f_beta_score

    return dict_scores



#     #Test above first
    
    
#     ##Compute competition metrics
    
#     # Step 4: Create a list of which tokens or subwords correspond to a word using the word_ids variable
#     word_subword_mapping = {}
#     for i, word_id in enumerate(word_ids):
#         if word_id is not None:
#             if word_id not in word_subword_mapping:
#                 word_subword_mapping[word_id] = []
#             word_subword_mapping[word_id].append(i)

#     # Step 5: Iterate through pairs of words and subwords to count the majority label
#     word_labels = []
#     for i, word in enumerate(words):
#         if i in word_subword_mapping:
#             subword_labels = pred_labels[word_subword_mapping[i]]
#             majority_label = get_majority(subword_labels)
#             word_labels.append((word, majority_label))
#         else:
#             word_labels.append((word,'O'))


#     token_v, pred_v = zip(*word_labels)

#     pred_conv = [model_id2cur_label[model.config.label2id[pred]] for pred in pred_v]
#     pred_bio = convert_to_bio(token_v, pred_conv)

#     f_beta = fbeta_score(labels, pred_bio, labels = classes_pos,beta=5, average='micro')
    
    
    
#     return metric.compute(predictions=predictions, references=labels)


    #Threshold + postprocessing
    #
    #Compute precision, recall, f1_beta from preds and labels
    
    #
    
    #Get 
    #Get

In [32]:
%%time 
compute_metrics = functools.partial(compute_metrics_base, model_adapter=model_adapter, threshold=0.18)
eval_pred = (loaded_predictions, labels_padded, None)

metrics_18 = compute_metrics((eval_pred)) 
metrics_18

CPU times: total: 1min 36s
Wall time: 1min 36s


{'f1': 0.27691285755801887,
 'precision': 0.16511535795623328,
 'recall': 0.8575469272306505,
 'f_beta': 0.738441361573116}

In [8]:
import functools

CPU times: total: 6.56 s
Wall time: 6.79 s


(6807, 3072)

In [ ]:
eval_dataset.select(range(2))

# Align Predictions

## Refactored

In [33]:
from collections import Counter

In [34]:
list_word_ids = [[word_id if word_id is not None else -100 for word_id in word_ids] for word_ids in tokenized_datasets['word_ids'] ]

word_ids_padded = pad_lists(list_word_ids, -100,  loaded_predictions.shape[1])

In [35]:
np_probs = loaded_predictions
true_labels_id = labels_padded
threshold = 0.18


#     logits, true_labels_id = eval_preds
#     np_probs = np.argmax(logits, axis=-1)
#     tokens = inputs.tokens()


label2id =  model_adapter.model.config.label2id
o_index = label2id['O']

np_sorted_indices = np.argsort(np_probs)
np_max_indices = np_sorted_indices[:,:, -1]
# np_max = np_sorted_indices[np.arange(np_max_indices.shape[0]), np_max_indices]
np_max_prob = np.take_along_axis(np_probs, np_max_indices[:, :, np.newaxis], axis=2).squeeze()

np_2nd_max_indices = np_sorted_indices[:, :, -2]
# np_2nd_max = np_sorted_indices[np.arange(np_sorted_indices.shape[0]), np_max_indices]
np_2nd_max_prob = np.take_along_axis(np_probs, np_2nd_max_indices[:, :, np.newaxis], axis=2).squeeze()

np_O_mask = np_max_indices == o_index
np_threshold_mask = np_2nd_max_prob > threshold

np_replace_mask = np_threshold_mask & np_O_mask

np_label_ids = np.where(np_replace_mask, np_2nd_max_indices, np_max_indices)


# Postprocess labels, convert irrelevant labels to 'O'
np_labels_irrelevant = np.array(model_adapter.labels_irrelevant)
label_ids_mask = np.isin(np_label_ids, np_labels_irrelevant)
np_label_ids[label_ids_mask] = o_index

flat_label_ids = np_label_ids.flatten()
flat_true_labels_id = true_labels_id.flatten()

# Remove padding preds for metrics
mask_padding_inv = flat_true_labels_id != -100
flat_true_labels_id = flat_true_labels_id[mask_padding_inv]
flat_label_ids = flat_label_ids[mask_padding_inv]


In [47]:
pred_tokens = np_label_ids

#Exclude beginning [CLS] Token 
#Also excludes end padding tokens by not appending row size (?)
diff_array = np.diff(word_ids_padded[:,1:], axis=1)

#Calculate row boundaries
non_zero = np.where(diff_array != 0, 1, 0)
group_ids = np.cumsum(non_zero, axis=1)
row_boundaries = np.cumsum(group_ids.max(axis = 1))

#Calculate splits, split into groups, Truncate > 10
split_indices =  np.where(diff_array.ravel() != 0)[0] + 1
list_groups = np.split(pred_tokens.ravel(), split_indices)
list_groups = [group[:10] if len(group) >= 10 else group for group in list_groups]

#30s Calculate modes per group
word_labels = [Counter(arr.tolist()).most_common(1)[0][0] if len(arr) > 0 else None for arr in list_groups]

#Fast (split into rows)
np_word_labels = np.array(word_labels)
list_pred_incomp = np.split(np_word_labels, row_boundaries)

#10s Initialize row arrays for word-level predictions
list_pred_words = [np.full((len(words)), model_adapter.O_id, dtype='int8') for words in tokenized_datasets['words']]

#5s #Get word_indices not skipped in word_ids
list_word_indices = []
for word_ids in tokenized_datasets['word_ids']:
    indices = set(word_ids)
    indices.discard(None)
    list_word_indices.append(np.array(list(indices)))
    
#200ms Align the processed word-level predictions to initialized array
for pred_words, word_indices, pred_incomp in zip(list_pred_words, list_word_indices, list_pred_incomp):
    pred_words[word_indices] = pred_incomp

## Latest

In [33]:
list_word_ids = [[word_id if word_id is not None else -100 for word_id in word_ids] for word_ids in tokenized_datasets['word_ids'] ]

word_ids_padded = pad_lists(list_word_ids, -100,  loaded_predictions.shape[1])

In [34]:
np_probs = loaded_predictions
true_labels_id = labels_padded
threshold = 0.18


#     logits, true_labels_id = eval_preds
#     np_probs = np.argmax(logits, axis=-1)
#     tokens = inputs.tokens()


label2id =  model_adapter.model.config.label2id
o_index = label2id['O']

np_sorted_indices = np.argsort(np_probs)
np_max_indices = np_sorted_indices[:,:, -1]
# np_max = np_sorted_indices[np.arange(np_max_indices.shape[0]), np_max_indices]
np_max_prob = np.take_along_axis(np_probs, np_max_indices[:, :, np.newaxis], axis=2).squeeze()

np_2nd_max_indices = np_sorted_indices[:, :, -2]
# np_2nd_max = np_sorted_indices[np.arange(np_sorted_indices.shape[0]), np_max_indices]
np_2nd_max_prob = np.take_along_axis(np_probs, np_2nd_max_indices[:, :, np.newaxis], axis=2).squeeze()

np_O_mask = np_max_indices == o_index
np_threshold_mask = np_2nd_max_prob > threshold

np_replace_mask = np_threshold_mask & np_O_mask

np_label_ids = np.where(np_replace_mask, np_2nd_max_indices, np_max_indices)

In [64]:
true_labels_id.dtype

dtype('int32')

In [176]:
word_ids_padded.shape

(6807, 3072)

In [178]:
pred_tokens = np_label_ids


#Exclude beginning [CLS] Token 
#Also excludes end padding tokens by not appending row size (?)
diff_array = np.diff(word_ids_padded[:,1:], axis=1)
# diff_array = np.diff(word_ids_padded, axis=1)

non_zero = np.where(diff_array != 0, 1, 0)
group_ids = np.cumsum(non_zero, axis=1)
row_boundaries = np.cumsum(group_ids.max(axis = 1))


split_indices =  np.where(diff_array.ravel() != 0)[0] + 1

In [179]:
diff_array

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 2, 1, ..., 0, 0, 0],
       [1, 2, 1, ..., 0, 0, 0],
       ...,
       [2, 1, 1, ..., 0, 0, 0],
       [0, 2, 1, ..., 0, 0, 0],
       [1, 2, 2, ..., 0, 0, 0]])

In [105]:
diff_array

array([[100,   1,   1, ...,   0,   0,   0],
       [100,   1,   2, ...,   0,   0,   0],
       [100,   1,   2, ...,   0,   0,   0],
       ...,
       [100,   2,   1, ...,   0,   0,   0],
       [100,   0,   2, ...,   0,   0,   0],
       [100,   1,   2, ...,   0,   0,   0]])

In [108]:
split_indices

array([       1,        2,        3, ..., 20901997, 20901998, 20901999],
      dtype=int64)

In [180]:
%%time
#12s
#pred_tokens[1:,:] = exclude first token ([CLS]
list_groups = np.split(pred_tokens.ravel(), split_indices)

CPU times: total: 11.7 s
Wall time: 12.3 s


In [181]:
%%time
list_groups = [group[:10] if len(group) >= 10 else group for group in list_groups]

CPU times: total: 906 ms
Wall time: 1.01 s


In [182]:
from collections import Counter

In [183]:
%%time
#30s
word_labels = [Counter(arr.tolist()).most_common(1)[0][0] if len(arr) > 0 else None for arr in list_groups]

CPU times: total: 25.9 s
Wall time: 26.6 s


In [184]:
%%time
np_word_labels = np.array(word_labels)

CPU times: total: 312 ms
Wall time: 330 ms


In [197]:
list_pred_incomp[-1]

array([17])

list_pred_incomp: 6808 | list_pred_words: 6807 | list_word_indices: 6807 | dataset: 6807


In [185]:
%%time
list_pred_incomp = np.split(np_word_labels, row_boundaries)

CPU times: total: 62.5 ms
Wall time: 61 ms


In [186]:
%%time
list_pred_words = [np.full((len(words)), model_adapter.O_id, dtype='int8') for words in tokenized_datasets['words']]

CPU times: total: 9.81 s
Wall time: 10.2 s


In [205]:
print(f'list_pred_incomp: {len(list_pred_incomp)} | list_pred_words: {len(list_pred_words)} | list_word_indices: {len(list_word_indices)} | dataset: {len(tokenized_datasets)}')

list_pred_incomp: 6808 | list_pred_words: 6807 | list_word_indices: 6807 | dataset: 6807


In [187]:
%%time
list_word_indices = []
for word_ids in tokenized_datasets['word_ids']:
    indices = set(word_ids)
    indices.discard(None)
    list_word_indices.append(np.array(list(indices)))

CPU times: total: 5.03 s
Wall time: 5.14 s


In [117]:
# %%time
# list_omit = []
# for i, (words, word_ids) in enumerate(zip(tokenized_datasets['words'], tokenized_datasets['word_ids'])):
#     max_index = len(words)
#     all_indices = set(range(max_index))
#     indices = set(word_ids)
#     remaining_indices = all_indices - indices
#     list_omit.append(remaining_indices)

CPU times: total: 15.2 s
Wall time: 15.7 s


In [122]:
# %%time
# list_word_indices = []
# for i, (words, word_ids) in enumerate(zip(tokenized_datasets['words'], tokenized_datasets['word_ids'])):
#     indices = set(word_ids)
#     indices.discard(None)
#     list_word_indices.append(np.array(list(indices)))

CPU times: total: 16.9 s
Wall time: 17.7 s


In [119]:
# %%time 
# for pred_words, word_indices, word_labels in zip(list_pred_words, list_word_indices, np_word_labels):
#     pred_words[word_indices] = np_word_labels

TypeError: 'numpy.int8' object does not support item assignment

In [209]:
%%time
for pred_words, word_indices, pred_incomp in zip(list_pred_words, list_word_indices, list_pred_incomp):
    pred_words[word_indices] = pred_incomp

CPU times: total: 188 ms
Wall time: 194 ms


In [95]:
list_word_indices[0]

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  12,  13,
        14,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  36,  37,  38,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
        56,  57,  58,  59,  60,  61,  62,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,
        97,  98,  99, 100, 102, 103, 104, 105, 106, 107, 108, 110, 111,
       112, 113, 114, 115, 117, 119, 120, 121, 122, 123, 124, 125, 126,
       127, 128, 129, 130, 131, 132, 133, 134, 136, 138, 140, 141, 142,
       144, 146, 147, 148, 149, 150, 151, 152, 154, 156, 157, 158, 159,
       160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172,
       173, 174, 176, 177, 179, 181, 182, 183, 184, 185, 186, 187, 188,
       189, 190, 191, 193, 195, 196, 197, 198, 199, 200, 202, 20

In [90]:
np.array(list_word_indices[0])

array({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102, 103, 104, 105, 106, 107, 108, 110, 111, 112, 113, 114, 115, 117, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 136, 138, 140, 141, 142, 144, 146, 147, 148, 149, 150, 151, 152, 154, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 176, 177, 179, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 193, 195, 196, 197, 198, 199, 200, 202, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 216, 218, 219, 221, 223, 224, 225, 226, 227, 229, 231, 232, 233, 234, 235, 236, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 25

In [ ]:
list_word_indices

In [125]:
list_pred_words

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [189]:
pred_words = tokenized_datasets['words'][0]

In [207]:
list_pred_words[0]

array([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 6, 6, 0, 6, 5,
       0, 5, 0, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [190]:
for pred_words, word_indices, pred_incomp in zip(list_pred_words, list_word_indices, list_pred_incomp):
    pred_words[word_indices] = pred_incomp

In [139]:
i_test = 4

print(f'{list_word_indices[i_test].shape} | {list_pred_incomp[i_test].shape}')

(1005,) | (1006,)


In [142]:
temp_words[752]

'\n\n'

In [202]:
print(f'temp_words: {len(temp_words)} | pred_words: {len(temp_words)} | word_indices: {len(word_indices)} | pred_incomp: {len(pred_incomp)}')

temp_words: 753 | pred_words: 753 | word_indices: 767 | pred_incomp: 767


In [166]:
print(f'temp_words: {len(temp_words)} | pred_words: {len(temp_words)} | word_indices: {len(word_indices)} | pred_incomp: {len(pred_incomp)}')

temp_words: 753 | pred_words: 753 | word_indices: 690 | pred_incomp: 691


In [168]:
temp_word_ids =  tokenized_datasets['word_ids'][0]

In [172]:
temp_word_counts = Counter(temp_word_ids)
temp_word_counts[None]

2

In [175]:
temp_word_ids[-1]

In [148]:
np_label_ids[0][0]

2

In [151]:
temp_subwords = tokenized_datasets['tokens'][0]

In [154]:
len(temp_subwords)

718

In [153]:
pred_words[0]

0

In [143]:
pred_words[752]

0

In [130]:
list_pred_words[0].shape

(753,)

In [140]:
list_word_indices[0]

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  12,  13,
        14,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  36,  37,  38,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
        56,  57,  58,  59,  60,  61,  62,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,
        97,  98,  99, 100, 102, 103, 104, 105, 106, 107, 108, 110, 111,
       112, 113, 114, 115, 117, 119, 120, 121, 122, 123, 124, 125, 126,
       127, 128, 129, 130, 131, 132, 133, 134, 136, 138, 140, 141, 142,
       144, 146, 147, 148, 149, 150, 151, 152, 154, 156, 157, 158, 159,
       160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172,
       173, 174, 176, 177, 179, 181, 182, 183, 184, 185, 186, 187, 188,
       189, 190, 191, 193, 195, 196, 197, 198, 199, 200, 202, 20

In [131]:
list_word_indices[0].shape

(690,)

In [133]:
list_pred_incomp[0].shape

(691,)

In [ ]:
f shape (691,) could not%%time 
for i in range(len(list_pred_words))
    list_pred_words[i][word_indices] = np_word_labels

In [74]:
set(range(10)) - set(range(5))

{5, 6, 7, 8, 9}

In [ ]:
omit_mask = 

In [ ]:
pred_words[] = pred_words_incomplete

In [55]:
len(pred_words)

6808

In [57]:
pred_words[-1]

array([0], dtype=object)

In [58]:
tokenized_datasets

Dataset({
    features: ['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels', 'input_ids', 'attention_mask', 'words', 'word_labels', 'word_ids'],
    num_rows: 6807
})

In [61]:
%%time
#10s
list_orig_lens = [len(words) for words in tokenized_datasets['words']]

CPU times: total: 10.4 s
Wall time: 10.6 s


In [62]:
list_orig_lens[0]

753

In [56]:
len(tokenized_datasets)

6807

In [54]:
pred_words

[array([None, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 6, 6, 6, 5, 0, 5, 0,
        5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [177]:
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])  # Example 2D array
B = np.array([0, 2, 5])  # Indices to split A after flattening, implying groups

# Step 1: Flatten A
A_flattened = A.flatten()

# Step 2 & 3: Split A_flattened using B and sum each group
# Calculate the indices for splitting
split_indices = np.diff(B, append=A_flattened.size)
# Use array_split to handle uneven splits
groups = np.array_split(A_flattened, np.cumsum(split_indices)[:-1])
group_sums = np.array([group.sum() for group in groups])

# Step 4: Map sums back to the original structure
# Initialize an array to hold the mapped sums
mapped_sums = np.zeros_like(A, dtype=float)

# Assuming each group maps back to each row in the original array
for i, sum_ in enumerate(group_sums):
    # Here, we simply assign the sum to all elements in the row
    # This assumes a direct mapping of groups to rows, which may need adjustment
    mapped_sums[i, :] = sum_

print("Original Array A:\n", A)
print("Group Sums:", group_sums)
print("Mapped Sums:\n", mapped_sums)

Original Array A:
 [[1 2 3]
 [4 5 6]
 [7 8 9]]
Group Sums: [ 3 12 30]
Mapped Sums:
 [[ 3.  3.  3.]
 [12. 12. 12.]
 [30. 30. 30.]]


In [180]:
import numpy as np

# Example data
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])  # 2D array
# Assuming B specifies splits within each row, adjusted for a 3x3 example
B = np.array([3, 6])  # Indices for splitting, ensuring at least one group per row

# Initialize an output array to store the sum for each row's groups
mapped_sums = np.zeros_like(A, dtype=float)

# Calculate row-wise splits and their sums
row_length = A.shape[1]
for i in range(A.shape[0]):
    row = A[i, :]
    # Adjust B for each row by subtracting the cumulative length of previous rows
    adjusted_B = B - i * row_length
    # Filter out indices not applicable for the current row
    valid_indices = adjusted_B[(adjusted_B > 0) & (adjusted_B <= row_length)]
    # Prepend 0 and append row_length to handle edge cases and ensure complete coverage of the row
    split_indices = np.concatenate(([0], valid_indices, [row_length]))
    # Split the row and sum each part
    row_groups = np.split(row, split_indices[1:-1])  # Exclude the artificially added edges
    row_sums = np.array([group.sum() for group in row_groups])
    # Assuming equal distribution of each group's sum across its originating cells in the row
    for j, sum_ in enumerate(row_sums):
        start_index = split_indices[j]
        end_index = split_indices[j+1]
        mapped_sums[i, start_index:end_index] = sum_

print("Original Array A:\n", A)
print("Mapped Sums:\n", mapped_sums)

Original Array A:
 [[1 2 3]
 [4 5 6]
 [7 8 9]]
Mapped Sums:
 [[ 6.  6.  6.]
 [15. 15. 15.]
 [24. 24. 24.]]


In [182]:
row_groups

[array([7, 8, 9])]

In [179]:
split_indices

array([2, 3, 4])

In [181]:
groups

[array([1, 2]), array([3, 4, 5]), array([6, 7, 8, 9])]

In [159]:
len(list_groups)

4613967

In [162]:
Counter([]).most_common(1)[0]

IndexError: list index out of range

In [170]:
pred_id_counts = Counter(word_labels).most_common()

In [171]:
pred_id_counts

[(17, 2945851),
 (0, 1193984),
 (2, 464377),
 (5, 3564),
 (6, 1797),
 (7, 1013),
 (14, 954),
 (12, 887),
 (9, 612),
 (1, 422),
 (4, 308),
 (16, 122),
 (8, 30),
 (10, 28),
 (11, 16),
 (None, 1),
 (15, 1)]

In [175]:
model_adapter.model.config.label2id

{'Biometric infomration': 17,
 'Civil state': 9,
 'Email address': 14,
 'Financial information': 16,
 'Genetic information': 15,
 'Health insurance information': 5,
 'Medical record numbers': 10,
 'Name': 2,
 'O': 0,
 'Philosophical beliefs': 13,
 'Phone Numbers': 4,
 'Physical addresses': 12,
 'Political opinions': 7,
 'Racial or ethnic information': 1,
 'Religious beliefs': 6,
 'Sexual orientation': 3,
 'Social Security number': 8,
 'Trade union membership': 11}

In [ ]:
%%time
# Example sorted array
# arr = np.array([1, 1, 2, 2, 2, 3, 4, 4, 5])
arr = word_ids_padded


# Find the indices where consecutive elements differ
diff_indices = list(np.where(np.diff(arr, axis=1) != 0))
diff_indices[1] += 1

# Append the length of the array to handle the last element
# diff_indices = np.append(diff_indices, arr.size)

# Count occurrences by computing differences
# Use np.insert to include the first element count
# counts = np.diff(np.insert(diff_indices, 0, 0), axis = 1)

# Extract unique values
# unique_vals = arr[diff_indices - 1]

In [307]:
non_zero

zero = 1 - non_zero

has_repeats = 1

In [ ]:
arr = np.array([1, 1, 2, 2, 2, 3, 4, 4, 5])
arr = word_ids_padded[0]

In [ ]:
arr[pad_mask] = 

In [ ]:
pad_mask = word_ids_padded[0] != -100

In [341]:
pad_mask.shape

(3072,)

In [340]:
diff_indices.shape

(692,)

In [391]:
diff_indices[0].shape

(4613966,)

In [392]:
diff_indices[1].shape

(4613966,)

In [388]:
np.insert(diff_indices, 0, 0, axis = 1)

array([[   0,    0,    0, ..., 6806, 6806, 6806],
       [   0,    1,    2, ...,  771,  772,  773]], dtype=int64)

In [385]:
counts = np.diff(np.insert(diff_indices, 0, 0, axis = 1), axis = 1)

In [387]:
counts.shape

(2, 4613966)

In [381]:
diff_indices[1]

1

In [378]:
diff_indices

array([   0,    1,    0, ..., 6806, 6806, 6806], dtype=int64)

## Calculate max word_len

In [82]:
%%time
# Example sorted array
# arr = np.array([1, 1, 2, 2, 2, 3, 4, 4, 5])

maxest = 0
highest_counts = None
# for i in range(1000):
for i in range(word_ids_padded.shape[0]):
    arr = word_ids_padded[i]


    # Find the indices where consecutive elements differ
    diff_indices = np.where(np.diff(arr) != 0)[0] + 1

    # Append the length of the array to handle the last element
    # diff_indices = np.append(diff_indices, arr.size)

    # Count occurrences by computing differences
    # Use np.insert to include the first element count
    counts = np.diff(np.insert(diff_indices, 0, 0))
    cur_max = counts.max()
    
    if cur_max > maxest:
        highest_counts = counts
        print(i)
    
    maxest = max(maxest, cur_max)

maxest

0
4
9
61
317
CPU times: total: 453 ms
Wall time: 479 ms


144

In [87]:
tokenized_datasets['words'][317]

['Cristina',
 'Valerio',
 '\n\n',
 'Design',
 'Tool',
 ':',
 'VISUALIZATION',
 '\n\n',
 'Challenge',
 '&',
 'Selection',
 '\n\n',
 'The',
 'tool',
 'I',
 'have',
 'decided',
 'to',
 'choose',
 'is',
 'Visualization',
 ',',
 'Visual',
 'Thinking',
 'as',
 'I',
 'prefer',
 'to',
 'call',
 'it',
 '.',
 ' ',
 'I',
 'consider',
 'this',
 'tool',
 'can',
 'be',
 'helpful',
 'to',
 'count',
 'on',
 'the',
 'whole',
 'picture',
 'of',
 'the',
 'problem',
 'I',
 'am',
 'working',
 'on',
 ',',
 'in',
 ' ',
 'the',
 'initial',
 'steps',
 'of',
 'the',
 'design',
 'thinking',
 'process',
 '.',
 'With',
 'it',
 ',',
 'I',
 'can',
 'support',
 'my',
 'effort',
 'to',
 'identify',
 'in',
 'the',
 'most',
 ' ',
 'accurate',
 'basis',
 ',',
 'the',
 'pain',
 'point',
 'I',
 'intend',
 'to',
 'solve',
 '.',
 '\n\n',
 'Visualization',
 'helps',
 'me',
 'to',
 'make',
 'the',
 'people',
 'pay',
 'attention',
 'to',
 'what',
 'is',
 'going',
 'on',
 '(',
 'What',
 'is',
 '?',
 ')',
 'by',
 'examining',
 ' 

In [83]:
highest_counts

array([  1,   1,   1,   1,   1,   1,   2,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   

In [51]:
counts

array([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 3, 1, 1, 3, 1, 1, 1, 1, 3,
       4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [53]:
%%time
# Example sorted array
# arr = np.array([1, 1, 2, 2, 2, 3, 4, 4, 5])
arr = word_ids_padded


# Find the indices where consecutive elements differ
diff_indices = list(np.where(np.diff(arr, axis=1) != 0))
diff_indices[1] += 1

# Append the length of the array to handle the last element
# diff_indices = np.append(diff_indices, arr.size)

# Count occurrences by computing differences
# Use np.insert to include the first element count
counts = np.diff(np.insert(diff_indices, 0, 0), axis = 1)

# Extract unique values
# unique_vals = arr[diff_indices - 1]

AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
COUNTS

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [67]:
np.diff(np.insert(diff_indices[1], 0, 0)).max()

144

In [60]:
diff_indices

[array([   0,    0,    0, ..., 6806, 6806, 6806], dtype=int64),
 array([  1,   2,   3, ..., 771, 772, 773], dtype=int64)]

In [70]:
diff_indices[1].max()

3071

In [71]:
diff_indices[1].argmax()

2227670

In [62]:
np.insert(diff_indices, 0, 0)

array([  0,   0,   0, ..., 771, 772, 773], dtype=int64)

In [55]:
counts = np.diff(np.insert(diff_indices, 0, 0))

In [59]:
counts.min()

-6805

In [393]:
arr = np.array([[0, 1, 1, 0],
                [1, 1, 0, 1],
                [0, 0, 1, 1]])

# Create a boolean mask of where arr is 1
mask = arr == 1

# Pad the mask with False on both sides horizontally
padded_mask = np.pad(mask, ((0, 0), (1, 1)), mode='constant', constant_values=False)

# Calculate differences horizontally, True where sequences start or end
diffs = np.diff(padded_mask.astype(int), axis=1)

# Positive differences indicate the start of a sequence, negative for the end
starts = diffs > 0
ends = diffs < 0

# Find the indices of starts and ends
start_indices = np.where(starts)
end_indices = np.where(ends)

# Calculate sequence lengths: exploit broadcasting to subtract start indices from end indices in a vectorized manner
# Ensuring the row indices match, then calculating the difference gives the sequence length
sequence_lengths = (end_indices[1] - start_indices[1]).reshape(-1, arr.shape[0])

print("Sequence Lengths:", sequence_lengths)

ValueError: cannot reshape array of size 4 into shape (3)

In [375]:
counts

array([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 3, 1, 1, 3, 1, 1, 1, 1, 3,
       4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
counts

In [355]:
counts_ref == counts

ValueError: operands could not be broadcast together with shapes (691,) (692,) 

In [364]:
np.where(np.diff(arr) == 0)[0].shape

(2380,)

In [369]:
arr[716:1000]

array([ 751, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
       -100, -100, -100, -100, -100, -100, -100, -1

In [370]:
diff_indices

array([   1,    2,    3,    4,    5,    7,    8,    9,   10,   11,   12,
         14,   15,   16,   17,   18,   19,   20,   21,   22,   23,   24,
         25,   26,   27,   28,   29,   30,   31,   32,   33,   34,   35,
         36,   37,   38,   39,   40,   41,   42,   43,   44,   45,   46,
         47,   48,   49,   50,   51,   53,   55,   56,   58,   60,   61,
         62,   63,   66,   67,   68,   71,   72,   73,   74,   75,   78,
         82,   83,   84,   85,   86,   87,   88,   89,   90,   91,   92,
         93,   94,   95,   96,   97,   98,   99,  100,  101,  102,  103,
        104,  105,  106,  107,  108,  109,  110,  112,  113,  114,  115,
        116,  117,  118,  120,  121,  122,  123,  124,  125,  126,  127,
        128,  129,  130,  131,  132,  133,  134,  135,  136,  137,  138,
        139,  140,  141,  142,  143,  144,  145,  146,  147,  148,  149,
        150,  151,  152,  153,  154,  155,  156,  157,  158,  159,  160,
        161,  162,  163,  164,  165,  166,  167,  1

In [367]:
np.diff(arr)[716:1000]

array([-851,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [352]:
counts_ref = counts

In [354]:
counts

array([   1,    1,    1,    1,    1,    2,    1,    1,    1,    1,    1,
          2,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    2,    2,    1,    2,    2,    1,
          1,    1,    3,    1,    1,    3,    1,    1,    1,    1,    3,
          4,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    2,    1,    1,    1,
          1,    1,    1,    2,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,   

In [345]:
np.diff(arr)[:1000]

array([ 100,    1,    1,    1,    1,    0,    1,    1,    1,    1,    1,
          1,    0,    2,    1,    1,    2,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    2,    1,    1,    2,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    0,    1,    0,    1,
          1,    0,    1,    0,    1,    1,    1,    1,    0,    0,    1,
          1,    2,    0,    0,    1,    1,    1,    1,    1,    0,    0,
          1,    0,    0,    0,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    2,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          0,    2,    1,    1,    1,    1,    1,    1,    0,    2,    1,
          1,    1,    1,    1,    2,    2,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    2,
          2,    2,    1,    1,    2,    2,    1,   

In [343]:
np.diff(arr).shape

(3071,)

In [339]:
diff_indices

array([   1,    2,    3,    4,    5,    7,    8,    9,   10,   11,   12,
         14,   15,   16,   17,   18,   19,   20,   21,   22,   23,   24,
         25,   26,   27,   28,   29,   30,   31,   32,   33,   34,   35,
         36,   37,   38,   39,   40,   41,   42,   43,   44,   45,   46,
         47,   48,   49,   50,   51,   53,   55,   56,   58,   60,   61,
         62,   63,   66,   67,   68,   71,   72,   73,   74,   75,   78,
         82,   83,   84,   85,   86,   87,   88,   89,   90,   91,   92,
         93,   94,   95,   96,   97,   98,   99,  100,  101,  102,  103,
        104,  105,  106,  107,  108,  109,  110,  112,  113,  114,  115,
        116,  117,  118,  120,  121,  122,  123,  124,  125,  126,  127,
        128,  129,  130,  131,  132,  133,  134,  135,  136,  137,  138,
        139,  140,  141,  142,  143,  144,  145,  146,  147,  148,  149,
        150,  151,  152,  153,  154,  155,  156,  157,  158,  159,  160,
        161,  162,  163,  164,  165,  166,  167,  1

In [338]:
pad_mask

array([False,  True,  True, ..., False, False, False])

In [328]:
counts

array([   1,    1,    1,    1,    1,    2,    1,    1,    1,    1,    1,
          2,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    2,    2,    1,    2,    2,    1,
          1,    1,    3,    1,    1,    3,    1,    1,    1,    1,    3,
          4,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    2,    1,    1,    1,
          1,    1,    1,    2,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,   

In [329]:
tokenized_datasets

Dataset({
    features: ['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels', 'input_ids', 'attention_mask', 'words', 'word_labels', 'word_ids'],
    num_rows: 6807
})

In [331]:
temp_attention_mask = tokenized_datasets['attention_mask'][0]

In [332]:
len(temp_attention_mask)

718

In [334]:
temp_attention_mask[-50:]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [318]:
diff_indices

array([2, 5, 6, 8, 9], dtype=int64)

In [319]:
 np.where(np.diff(arr) != 0)[0] + 1

array([2, 5, 6, 8], dtype=int64)

In [317]:
counts

array([2, 3, 1, 2, 1], dtype=int64)

In [310]:
zero

array([[0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       ...,
       [0, 0, 0, ..., 1, 1, 1],
       [0, 1, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1]])

In [311]:
%%time
mask = zero == 0

# get classical cumsum
cs = zero.cumsum(axis=1)

# ffill the cumsum value on 1s
# subtract from cumsum
out = cs - np.maximum.accumulate(np.where(mask, cs, 0), axis=1)

CPU times: total: 312 ms
Wall time: 327 ms


In [327]:
out[1]

array([   0,    0,    0, ..., 2545, 2546, 2547])

In [301]:
diff_array

array([[100,   1,   1, ...,   0,   0,   0],
       [100,   1,   2, ...,   0,   0,   0],
       [100,   1,   2, ...,   0,   0,   0],
       ...,
       [100,   2,   1, ...,   0,   0,   0],
       [100,   0,   2, ...,   0,   0,   0],
       [100,   1,   2, ...,   0,   0,   0]])

In [312]:
np.max(out, axis = 1)

array([2354, 2547, 2390, ..., 2474, 2424, 2298])

In [300]:
out[300:]

array([[  100,   101,     0, ...,  -765,  -765,  -765],
       [  100,   101,   102, ..., -1282, -1282, -1282],
       [  100,   101,   102, ...,  -723,  -723,  -723],
       ...,
       [  100,   102,   103, ...,  -439,  -439,  -439],
       [  100,     0,     2, ...,  -736,  -736,  -736],
       [  100,   101,   103, ...,  -350,  -350,  -350]])

In [289]:
out

array([[1, 2, 3, ..., 0, 0, 0],
       [1, 2, 3, ..., 0, 0, 0],
       [1, 2, 3, ..., 0, 0, 0],
       ...,
       [1, 2, 3, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 2, 3, ..., 0, 0, 0]])

In [287]:
diff_array

array([[100,   1,   1, ...,   0,   0,   0],
       [100,   1,   2, ...,   0,   0,   0],
       [100,   1,   2, ...,   0,   0,   0],
       ...,
       [100,   2,   1, ...,   0,   0,   0],
       [100,   0,   2, ...,   0,   0,   0],
       [100,   1,   2, ...,   0,   0,   0]])

In [278]:
diff_array

array([[100,   1,   1, ...,   0,   0,   0],
       [100,   1,   2, ...,   0,   0,   0],
       [100,   1,   2, ...,   0,   0,   0],
       ...,
       [100,   2,   1, ...,   0,   0,   0],
       [100,   0,   2, ...,   0,   0,   0],
       [100,   1,   2, ...,   0,   0,   0]])

In [286]:
out[0][600:800]

array([103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
       116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
       142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
       155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
       168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180,
       181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193,
       194, 195, 196, 197, 198, 199, 200, 201, 202,   0,   1,   2,   3,
         4,   5,   6,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [276]:
max_lens = out.max(axis = 1)

In [277]:
max_lens

array([202, 330, 485, ..., 275, 291, 533])

In [273]:
out

array([[1, 2, 3, ..., 0, 0, 0],
       [1, 2, 3, ..., 0, 0, 0],
       [1, 2, 3, ..., 0, 0, 0],
       ...,
       [1, 2, 3, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 2, 3, ..., 0, 0, 0]])

In [241]:
word_ids_padded[0][:300]

array([-100,    0,    1,    2,    3,    4,    4,    5,    6,    7,    8,
          9,   10,   10,   12,   13,   14,   16,   17,   18,   19,   20,
         21,   22,   23,   24,   25,   26,   27,   28,   29,   30,   31,
         32,   33,   34,   36,   37,   38,   40,   41,   42,   43,   44,
         45,   46,   47,   48,   49,   50,   51,   52,   52,   53,   53,
         54,   55,   55,   56,   56,   57,   58,   59,   60,   60,   60,
         61,   62,   64,   64,   64,   65,   66,   67,   68,   69,   69,
         69,   70,   70,   70,   70,   71,   72,   73,   74,   75,   76,
         77,   78,   79,   80,   81,   82,   83,   84,   86,   87,   88,
         89,   90,   91,   92,   93,   94,   95,   96,   97,   98,   99,
        100,  100,  102,  103,  104,  105,  106,  107,  108,  108,  110,
        111,  112,  113,  114,  115,  117,  119,  120,  121,  122,  123,
        124,  125,  126,  127,  128,  129,  130,  131,  132,  133,  134,
        136,  138,  140,  141,  142,  144,  146,  1

In [244]:
non_zero[0][:300]

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [257]:
arr_in = np.array([1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1])

mask = arr_in == 0

# get classical cumsum
cs = arr_in.cumsum(axis=1)

# ffill the cumsum value on 1s
# subtract from cumsum
out = cs - np.maximum.accumulate(np.where(mask, cs, 0), axis=1)

AxisError: axis 1 is out of bounds for array of dimension 1

In [259]:
arr_in = np.array([1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1])

mask = arr_in == 0

# get classical cumsum
cs = arr_in.cumsum()

# ffill the cumsum value on 1s
# subtract from cumsum
out = cs - np.maximum.accumulate(np.where(mask, cs, 0))

In [266]:
out

array([1, 2, 0, 1, 2, 0, 1, 2, 3, 0, 0, 1])

In [264]:
cs

array([1, 2, 2, 3, 4, 4, 5, 6, 7, 7, 7, 8])

In [268]:
mask

array([False, False,  True, False, False,  True, False, False, False,
        True,  True, False])

In [269]:
np.where(mask, cs, 0)

array([0, 0, 2, 0, 0, 4, 0, 0, 0, 7, 7, 0])

In [260]:
out

array([1, 2, 0, 1, 2, 0, 1, 2, 3, 0, 0, 1])

In [250]:
arr = np.array([1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1])

In [270]:
arr.cumsum() - arr.cumsum().where(~arr).ffill().fillna(0).astype(int)

AttributeError: 'numpy.ndarray' object has no attribute 'where'

In [238]:
diff_array[0]

array([100,   1,   1, ...,   0,   0,   0])

In [232]:
#From word_ids padded
diff_array[diff_array == 100] = -100
#Get diff (i+1 - i)

#Get max per row
max_wordlen = diff_array.max(axis=1)
#that max will be the padding limit


max_wordlen


array([2, 2, 2, ..., 2, 2, 2])

In [235]:
diff_array.max()

100

In [229]:
diff_array

array([[-100,    1,    1, ...,    0,    0,    0],
       [-100,    1,    2, ...,    0,    0,    0],
       [-100,    1,    2, ...,    0,    0,    0],
       ...,
       [-100,    2,    1, ...,    0,    0,    0],
       [-100,    0,    2, ...,    0,    0,    0],
       [-100,    1,    2, ...,    0,    0,    0]])

In [224]:
diff_array = diff_array.max(axis=1)

In [221]:
max_wordlen = diff_array.max(axis=1)

In [222]:
max_wordlen

array([100, 100, 100, ..., 100, 100, 100])

In [168]:
split_indices =  np.where(diff_array.ravel() != 0)[0]

In [167]:
split_indices

(array([       0,        1,        2, ..., 20901996, 20901997, 20901998],
       dtype=int64),)

In [219]:
np.diff(split_indices).max()

3006

## Groups and modes

In [192]:
%%time
list_groups = np.split(pred_tokens.ravel(), split_indices)

CPU times: total: 11.2 s
Wall time: 11.8 s


In [193]:
len(list_groups)

4613967

In [209]:
%%time
modes = [stats.mode(group)[0] for group in list_groups[:10000]]

CPU times: total: 4.61 s
Wall time: 4.8 s


In [215]:
%%time
modes = [Counter(arr.tolist()).most_common(1) for arr in list_groups[:100000]]

CPU times: total: 3.66 s
Wall time: 9.58 s


### Test Scipy mode

In [133]:
from scipy import stats

np_rand = np.random.randint(0, high=20, size=(100000,3000), dtype=int)

In [93]:
np_rand

array([[ 4, 16, 19, ...,  5,  0,  3],
       [ 2, 18, 12, ..., 16, 14,  6],
       [ 9,  7, 11, ..., 11,  3, 13],
       ...,
       [19, 12, 16, ..., 14,  3,  6],
       [ 9,  2, 16, ..., 17,  9, 18],
       [ 5, 14,  3, ..., 18, 10, 11]])

In [140]:
np_rand_short = np.random.randint(0, high=20, size=(100000,144), dtype=int)
np_rand_shortest = np.random.randint(0, high=20, size=(5000000,10), dtype=int)

In [103]:
np_rand_short.shape

(100000, 144)

In [134]:
%%time
np_modes = stats.mode(np_rand, axis = 1)

CPU times: total: 16.8 s
Wall time: 17.2 s


In [102]:
%%time
np_modes = stats.mode(np_rand_short, axis = 1)

CPU times: total: 9.09 s
Wall time: 9.58 s


In [107]:
%%time
np_modes = stats.mode(np_rand_shortest, axis = 1)

CPU times: total: 8.75 s
Wall time: 9.19 s


In [136]:
%%time

modes = np.apply_along_axis(stats.mode, axis=1, arr=np_rand_shortest)

CPU times: total: 47.4 s
Wall time: 49.3 s


In [135]:
%%time
from collections import Counter
modes = np.apply_along_axis(lambda arr: Counter(arr.tolist()).most_common(1)[0][0], axis = 1, arr =  np_rand)

CPU times: total: 25.1 s
Wall time: 26.4 s


In [141]:
%%time
from collections import Counter
modes = np.apply_along_axis(lambda arr: Counter(arr.tolist()).most_common(1)[0][0], axis = 1, arr =  np_rand_shortest)

CPU times: total: 56.9 s
Wall time: 59.4 s


In [127]:
modes.shape

(100000,)

In [114]:
modes.shape

(100000, 1, 2)

In [ ]:
np_rand_shortest

In [ ]:
# modes = [stats.mode(group)[0] for group in list_groups[:10000]]

In [ ]:
modes = 

In [106]:
np_rand_shortest.shape

(100000, 10)

# Experiments

In [40]:
import numpy as np

# Example data
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])  # 2D array
# Example split indices for each row
B = np.array([3, 6])  # Indices for splitting, ensuring at least one group per row

# Split A into groups
groups = np.split(A.flatten(), B)

# Calculate row-wise sums for each group
group_sums = [group.sum(axis=1) for group in groups]

# Map sums back to original rows of A
mapped_sums = []
row_idx = 0
for row in group_sums:
    mapped_sum_row = np.zeros_like(A[row_idx])
    for group in row:
        mapped_sum_row[:len(group)] += group
    mapped_sums.append(mapped_sum_row)
    row_idx += 1

print("Original Array A:\n", A)
print("Mapped Sums:\n", mapped_sums)

AxisError: axis 1 is out of bounds for array of dimension 1

In [39]:
row_boundaries

array([0, 3, 6, 9])

# Bin

In [ ]:
%%time

def pad_lists(lst, pad_value, max_length = None):
    
    if not max_length:
        max_length = max(len(sublist) for sublist in lst)
    padded_lst = [sublist + [pad_value] * (max_length - len(sublist)) for sublist in lst]
    return np.array(padded_lst)

# Example list of lists with varying lengths
list_of_lists = [[1, 2, 3], [4, 5], [6], [7, 8, 9, 10]]

# Padding value
pad_value = 0

padded_array = pad_lists(tokenized_datasets['labels'], pad_value)
padded_array.shape

In [ ]:
entities = ['O', 'Email' 'ID_NUM', 'NAME_STUDENT', 'PHONE_NUM', 'STREET_ADDRESS', 'URL_PERSONAL', 'USERNAME']
entity_id2class_id = {entity:[f'B_{}',f'I_{}'] for entity in entities}

entity_id2class_id['O'] = ['O','O']
entity_id2class_id['Email'][1] = entity_id2class_id['Email'][2]
entity_id2class_id['ID_NUM'][1] = entity_id2class_id['ID_NUM'][2]



In [ ]:
#Pad word ids
#Group label ids that have the same word_id
#word-level label =  most frequent label
#Convert to BIO


pred_tokens = loaded_predictions


word_boundaries = np.where(np.diff(word_ids_padded) != 0)[0] + 1
word_groups = np.split(pred_tokens, word_boundaries)


pred_labels = [stats.mode(group)[0] for group in word_groups]

#Get repetition boundaries
pred_boundaries = np.where(np.diff(pred_boundaries) != 0)[0] + 1

#Create [B-, I-] Mapping for every element]
entity_id2class_id

#If in boundary - asign as B- (index 0)
#Else use I-

word_labels_bio = np.where(pred_boundaries != 0, entity_id2class_id[A,0], entity_id2class_id[A,1])

